In [408]:
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer, CountVectorizer
import glob
import os 
import pandas as pd
import datetime
from scipy.stats import iqr
import numpy as np

from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split

from sklearn.pipeline import FeatureUnion
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

spm = None
english_words = None
excepted_stock_names = []
limit = 20

In [409]:
import keras
from keras.layers import Dense, Dropout, RepeatVector, BatchNormalization, Convolution1D, Flatten, Lambda, Permute, MaxPooling1D, AlphaDropout
from keras.models import Sequential
from keras.utils import to_categorical
import keras.backend as K
from sklearn.model_selection import train_test_split
from keras.models import load_model

In [410]:
import pickle

In [411]:
# pickle.dump(obj, file, protocol=None, *, fix_imports=True)
# pickle.load(file, *, fix_imports=True, encoding="ASCII", errors="strict")¶
class StockPriceMapping:
    def __init__(self, limit=-1):
        self.limit = limit
        self.mapping = self.get_mapping()
        
    def get_mapping(self):
        stock_price_mapping = {}
        for file in glob.glob('stanford_nlp_sec/8k-gz/*')[0:self.limit]:
            stock_name = file.split('/')[-1]
            file_data = open(file, 'r').read()
            time_data = ""
            try:
                with open(file) as open_file:
                    file_data = [next(open_file) for x in range(3)]
                file_data = ''.join(file_data)
                time_data = file_data.split("\n")[2][5:14]
            except:
                print("No data for ", stock_name)
                excepted_stock_names.append(stock_name)
                continue
            year = time_data[0:4]
            month = time_data[4:6]
            day = time_data[6:8]
            date_stamp = ("%s-%s-%s" %(year, month, day))
            stock_price_mapping[stock_name] = {'date': date_stamp}
        return stock_price_mapping

In [412]:
class PriceData:
    def __init__(self, stock_name):
        self.stock_name = stock_name
        self.price_data = pd.read_csv('stanford_nlp_sec/price_history/' + stock_name + '.csv')

    def on_date(self, date, market_time = 'Open'): 
        try:
            return float(self.price_data.loc[self.price_data['Date'] == date][market_time])
        except: 
            return None


In [413]:
class FilenamesToStockNamesTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        return None
    
    def fit(self, X, y=None):
        return self
    
    def fit_transform(self, X, y=None):
        return self.fit(X, y).transform(X)
        
    def transform(self, X):
        output = []
        for filename in X:
            stock_name = filename.split('/')[-1]
            output.append(stock_name)
        return output
class MapStockNamesToDatesTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, start_int, end_int):
        self.start_int = start_int
        self.end_int = end_int
        self.range = (range(start_int, end_int))
    
    def fit(self, X, y=None):
        return self
    
    def fit_transform(self, X, y=None):
        return self.fit(X, y).transform(X)
        
    def transform(self, X):
        output = {}
        for stock_name in X:
            try:
                date = spm.mapping[stock_name]['date']
            except: 
                continue # Ignore stocks which don't have a date
            dates = []
            for delta in self.range:
                date_delta = datetime.timedelta(days=delta)
                date_string = datetime.datetime.strptime(date, '%Y-%m-%d').date()
                dates.append(str(date_string + date_delta))
            output[stock_name] = dates
        return output
class StockNameDatesMapToPricesListTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        return None
    
    def fit(self, X, y=None):
        return self
    
    def fit_transform(self, X, y=None):
        return self.fit(X, y).transform(X)
        
    def transform(self, X):
        output = []
        for stock_name in X.keys():
            prices = []
            for date in X[stock_name]:
                prices.append(PriceData(stock_name).on_date(date, 'Close'))
            output.append(prices)
        return np.array(output)
class LabelsTransform(BaseEstimator, TransformerMixin):
    # Returns the interquartile-range and median.
    def __init__(self):
        return None
        
    def fit(self, X, y=None):
        # ldcom = last_day_change_over_median
        self.ldcom = []
        for prices in X:
            this_median = np.median(prices[0:-3])
            self.ldcom.append(((prices[-1]-this_median)/this_median))
        return self
    
    def fit_transform(self, X, y=None):
        self.fit(X, y)
        
        return np.array(self.ldcom)
        
    def transform(self, X):
        return self
            
class StatisticalMeasuresTransformer(BaseEstimator, TransformerMixin):
    # Returns the interquartile-range and median.
    def __init__(self):
        return None
        
    def fit(self, X, y=None):
        # ldcom = last_day_change_over_median
        
        output = []
        self.iqr_var = []
        self.median = []
        for prices in X:
            this_iqr = iqr(prices[0:-3])
            this_median = np.median(prices[0:-3])
            self.iqr_var.append(this_iqr)
            self.median.append(this_median)
        return self
    
    def fit_transform(self, X, y=None):
        return self.fit(X, y).transform(X)
        
    def transform(self, X):
        X_output = []
        for i, prices in enumerate(X):
            stats   = []
            iqr_var = self.iqr_var[i] or iqr(prices)
            median  = self.median[i]  or np.median(prices)
            
            stats.append(iqr_var)
            stats.append(median)
            
            X_output.append(stats)
        return np.array(X_output)
class SparseToArray(BaseEstimator, TransformerMixin):
    def __init__(self):
        return None
        
    def fit(self, X, y=None):
        return self
    
    def fit_transform(self, X, y=None):
        return self.fit(X, y).transform(X)
        
    def transform(self, X):
        return X.toarray() #[ar.toarray() for ar in X]
    
class ReadFiles(BaseEstimator, TransformerMixin):
    def __init__(self):
        return None
        
    def fit(self, X, y=None):
        return self
    
    def fit_transform(self, X, y=None):
        return self.fit(X, y).transform(X)
        
    def transform(self, X):
        return (open(filename, 'r').read() for filename in X)  
class CustomCountVectorizer(BaseEstimator, TransformerMixin):
    def __init__(self, vocabulary = None):
        self.dictionary = vocabulary or open('/usr/share/dict/words', 'r').read().split("\n")
        
    def fit(self, X, y=None):
        return self
    
    def fit_transform(self, X, y=None):
        return self.fit(X, y).transform(X)
        
    def transform(self, X):
        output = []
        for file_contents in X: 
            feature = np.zeros(len(self.dictionary))
            ar = file_contents.split(' ')
            ar = [re.sub("<|>", " ", b.lower()) for b in ar]
            for word in ar:
                try:
                    index = self.dictionary.index(word)
                    feature[index] = feature[index] + 1
                except:
                    continue
            output.append(feature)
        return output
class DocLengthNormalizer(BaseEstimator, TransformerMixin):
    def __init__(self):
        return None 
    
    def fit(self, X, y=None):
        return self
    
    def fit_transform(self, X, y=None):
        return self.fit(X, y).transform(X)
        
    def transform(self, X):
        output = []
        for feature in X: 
            new_feature = np.zeros(len(feature))
            feature_sum = float(sum(feature))
            for i,fi in enumerate(feature):
                new_feature[i] = fi/feature_sum
            output.append(new_feature)
        return output
class CustomTfIdf(BaseEstimator, TransformerMixin):
    def __init__(self):
        return None 
    
    def fit(self, X, y=None):
        self.column_averages = np.sum(X, axis=0)/len(X)
        return self
    
    def fit_transform(self, X, y=None):
        return self.fit(X, y).transform(X)
        
    def transform(self, X):
        output = []
        for feature in X: 
            new_feature = np.zeros(len(feature))
            for i,fi in enumerate(feature):
                new_feature[i] = (fi/self.column_averages[i])
            output.append(new_feature)
        return output
class ToNpArray(BaseEstimator, TransformerMixin):
    def __init__(self):
        return None 
    
    def fit(self, X, y=None):
        return self
    
    def fit_transform(self, X, y=None):
        return self.fit(X, y).transform(X)
        
    def transform(self, X):
        return np.array(X)
            

In [414]:
def get_filenames(limit = -1):
    filenames = []
    directory_files = glob.glob('stanford_nlp_sec/8k-gz/*')
    excepted_files = [('stanford_nlp_sec/8k-gz/' + sn) for sn in excepted_stock_names]
    filenames_with_data = [x for x in directory_files[:limit] if x not in excepted_files]
    for filename in filenames_with_data:
        filenames.append(filename)
    return filenames

In [415]:
spm = spm or StockPriceMapping(limit)

filenames = get_filenames(limit)


# Used to build other pipelines
prices_pipeline = Pipeline([
    ('filenames_to_stock_names', FilenamesToStockNamesTransformer()),
    ('stock_names_to_dates', MapStockNamesToDatesTransformer(-30, 2)),
    ('dates_to_prices_transformer', StockNameDatesMapToPricesListTransformer()),
    ('imputer', SimpleImputer())
])

# Used as the final y values 
labels_pipeline = Pipeline([
    ('prices_pipeline', prices_pipeline),
    ('labels_transform', LabelsTransform())
])

# Used for training and test set features
stock_stats = Pipeline([
    ('prices_pipeline', prices_pipeline),
    ('stats_transform', StatisticalMeasuresTransformer())
])

custom_vocabulary = ['lawsuit', 'lawyer', 'firm', 'split', 'report', 'intended', 'seed', 'weak', 'increase', 'growth', 'new', 'strong', 'forward', 'well', 'grow', 'product', 'future', 'we', 'charge', 'loss', 'lower', 'decline', 'down', 'reduce', 'layoff', 'adjust', 'regulation', 'offset', 'reduction', 'while']
# CountVectorizer can take vocabulary=custom_vocabulary if needed.
# Used for training and test set features 
text_word_counts = Pipeline([
    ('read_files', ReadFiles()),
    ('vect', CountVectorizer(
                token_pattern=r"[a-zA-Z]+", 
                stop_words = 'english', 
                min_df = 0.9,
                max_features=None, 
                ngram_range=(1, 5))),
    ('sparse_to_array', SparseToArray()),
    ('doc_length_normalizer', DocLengthNormalizer()),
    ('tf_idf', CustomTfIdf()),
    ('np_array', ToNpArray())
])

# token_pattern =
# vectorizer = CountVectorizer
# vectorizer.fit_transform(get_filenames(3)).toarray()

feature_union = FeatureUnion(transformer_list=[
    ('text_tf_idf', text_word_counts),
    ('stats_features', stock_stats),
    ('std_scaler', StandardScaler()),
])

full_pipeline = Pipeline([
    ('full_pipeline', feature_union)
])

In [416]:
# stock_stats # text_word_counts
X = text_word_counts.fit_transform(get_filenames(limit))
X.shape

(20, 4673)

In [417]:
y = labels_pipeline.fit_transform(get_filenames(limit))

In [418]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [433]:
clf = RandomForestRegressor(n_estimators=200, max_depth=15, verbose=1, n_jobs=2)
clf.fit(X_train, y_train)

[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    1.9s
[Parallel(n_jobs=2)]: Done 200 out of 200 | elapsed:    1.9s finished


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=15,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=2,
           oob_score=False, random_state=None, verbose=1, warm_start=False)

In [434]:
print(clf.score(X_train, y_train))
print(clf.score(X_test, y_test))

[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 200 out of 200 | elapsed:    0.0s finished
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 200 out of 200 | elapsed:    0.1s finished


0.837980637989
-0.834684182067


In [439]:
shape = X_train.shape[1]

m1 = Sequential([
    BatchNormalization(input_shape=(shape,)),
    Dense(80, activation='relu'),
    Dropout(0.5),
    BatchNormalization(),
    Dense(40, activation='relu'),
    Dropout(0.5),
    BatchNormalization(),
    Dense(20, activation='relu'),
    Dropout(0.5),
    BatchNormalization(),
    Dense(1)   
])

m1.compile(optimizer='rmsprop',
              loss='mean_absolute_error',
              metrics=['mae'])

sched = [[0.0001, 2], [0.001, 20], [0.01, 2], [0.1, 2], [0.5, 1], [0.1, 5], [0.01, 20], [0.001, 40], [0.0001, 80], [0.00005, 120]]

for i in range(5):
    for info in sched:
        lr, epochs = info
        m1.optimizer.lr = lr
        m1.fit(np.array(X_train), np.array(y_train), epochs=epochs,  batch_size=64, validation_data=(np.array(X_test), np.array(y_test)))

Train on 16 samples, validate on 4 samples
Epoch 1/2
16/16 [==============================] - 1s 32ms/step - loss: 0.9705 - mean_absolute_error: 0.9705 - val_loss: 1.4414 - val_mean_absolute_error: 1.4414
Epoch 2/2
16/16 [==============================] - 0s 986us/step - loss: 1.1381 - mean_absolute_error: 1.1381 - val_loss: 1.4089 - val_mean_absolute_error: 1.4089
Train on 16 samples, validate on 4 samples
Epoch 1/20
16/16 [==============================] - 0s 907us/step - loss: 1.1989 - mean_absolute_error: 1.1989 - val_loss: 1.3833 - val_mean_absolute_error: 1.3833
Epoch 2/20
16/16 [==============================] - 0s 1ms/step - loss: 1.1579 - mean_absolute_error: 1.1579 - val_loss: 1.3845 - val_mean_absolute_error: 1.3845
Epoch 3/20
16/16 [==============================] - 0s 1ms/step - loss: 0.8883 - mean_absolute_error: 0.8883 - val_loss: 1.3524 - val_mean_absolute_error: 1.3524
Epoch 4/20
16/16 [==============================] - 0s 1ms/step - loss: 1.1285 - mean_absolute_error:

Epoch 18/20
16/16 [==============================] - 0s 1ms/step - loss: 0.8957 - mean_absolute_error: 0.8957 - val_loss: 0.6525 - val_mean_absolute_error: 0.6525
Epoch 19/20
16/16 [==============================] - 0s 2ms/step - loss: 1.2103 - mean_absolute_error: 1.2103 - val_loss: 0.6545 - val_mean_absolute_error: 0.6545
Epoch 20/20
16/16 [==============================] - 0s 1ms/step - loss: 1.1627 - mean_absolute_error: 1.1627 - val_loss: 0.6530 - val_mean_absolute_error: 0.6530
Train on 16 samples, validate on 4 samples
Epoch 1/40
16/16 [==============================] - 0s 1ms/step - loss: 1.4973 - mean_absolute_error: 1.4973 - val_loss: 0.6355 - val_mean_absolute_error: 0.6355
Epoch 2/40
16/16 [==============================] - 0s 2ms/step - loss: 1.0294 - mean_absolute_error: 1.0294 - val_loss: 0.6346 - val_mean_absolute_error: 0.6346
Epoch 3/40
16/16 [==============================] - 0s 1ms/step - loss: 1.1585 - mean_absolute_error: 1.1585 - val_loss: 0.6313 - val_mean_absol

Epoch 8/80
16/16 [==============================] - 0s 1ms/step - loss: 0.8265 - mean_absolute_error: 0.8265 - val_loss: 0.4809 - val_mean_absolute_error: 0.4809
Epoch 9/80
16/16 [==============================] - 0s 1ms/step - loss: 1.0290 - mean_absolute_error: 1.0290 - val_loss: 0.4828 - val_mean_absolute_error: 0.4828
Epoch 10/80
16/16 [==============================] - 0s 1ms/step - loss: 0.8878 - mean_absolute_error: 0.8878 - val_loss: 0.4553 - val_mean_absolute_error: 0.4553
Epoch 11/80
16/16 [==============================] - 0s 2ms/step - loss: 0.9138 - mean_absolute_error: 0.9138 - val_loss: 0.4543 - val_mean_absolute_error: 0.4543
Epoch 12/80
16/16 [==============================] - 0s 1ms/step - loss: 1.0702 - mean_absolute_error: 1.0702 - val_loss: 0.4415 - val_mean_absolute_error: 0.4415
Epoch 13/80
16/16 [==============================] - 0s 1ms/step - loss: 1.0507 - mean_absolute_error: 1.0507 - val_loss: 0.4294 - val_mean_absolute_error: 0.4294
Epoch 14/80
16/16 [=====

Epoch 59/80
16/16 [==============================] - 0s 1ms/step - loss: 0.7917 - mean_absolute_error: 0.7917 - val_loss: 0.3969 - val_mean_absolute_error: 0.3969
Epoch 60/80
16/16 [==============================] - 0s 945us/step - loss: 0.9013 - mean_absolute_error: 0.9013 - val_loss: 0.3869 - val_mean_absolute_error: 0.3869
Epoch 61/80
16/16 [==============================] - 0s 1ms/step - loss: 0.8326 - mean_absolute_error: 0.8326 - val_loss: 0.3750 - val_mean_absolute_error: 0.3750
Epoch 62/80
16/16 [==============================] - 0s 1ms/step - loss: 0.8727 - mean_absolute_error: 0.8727 - val_loss: 0.3936 - val_mean_absolute_error: 0.3936
Epoch 63/80
16/16 [==============================] - 0s 1ms/step - loss: 0.8746 - mean_absolute_error: 0.8746 - val_loss: 0.3959 - val_mean_absolute_error: 0.3959
Epoch 64/80
16/16 [==============================] - 0s 1ms/step - loss: 0.7698 - mean_absolute_error: 0.7698 - val_loss: 0.3848 - val_mean_absolute_error: 0.3848
Epoch 65/80
16/16 [=

Epoch 29/120
16/16 [==============================] - 0s 1ms/step - loss: 1.1502 - mean_absolute_error: 1.1502 - val_loss: 0.3233 - val_mean_absolute_error: 0.3233
Epoch 30/120
16/16 [==============================] - 0s 1ms/step - loss: 0.8503 - mean_absolute_error: 0.8503 - val_loss: 0.3313 - val_mean_absolute_error: 0.3313
Epoch 31/120
16/16 [==============================] - 0s 1ms/step - loss: 1.3266 - mean_absolute_error: 1.3266 - val_loss: 0.3389 - val_mean_absolute_error: 0.3389
Epoch 32/120
16/16 [==============================] - 0s 1ms/step - loss: 0.7171 - mean_absolute_error: 0.7171 - val_loss: 0.3409 - val_mean_absolute_error: 0.3409
Epoch 33/120
16/16 [==============================] - 0s 958us/step - loss: 1.0066 - mean_absolute_error: 1.0066 - val_loss: 0.3324 - val_mean_absolute_error: 0.3324
Epoch 34/120
16/16 [==============================] - 0s 1ms/step - loss: 1.0014 - mean_absolute_error: 1.0014 - val_loss: 0.3318 - val_mean_absolute_error: 0.3318
Epoch 35/120
1

...

Epoch 30/40
16/16 [==============================] - 0s 1ms/step - loss: 0.5981 - mean_absolute_error: 0.5981 - val_loss: 0.1148 - val_mean_absolute_error: 0.1148
Epoch 31/40
16/16 [==============================] - 0s 1ms/step - loss: 0.7596 - mean_absolute_error: 0.7596 - val_loss: 0.1089 - val_mean_absolute_error: 0.1089
Epoch 32/40
16/16 [==============================] - 0s 2ms/step - loss: 0.5426 - mean_absolute_error: 0.5426 - val_loss: 0.1124 - val_mean_absolute_error: 0.1124
Epoch 33/40
16/16 [==============================] - 0s 2ms/step - loss: 0.9006 - mean_absolute_error: 0.9006 - val_loss: 0.1133 - val_mean_absolute_error: 0.1133
Epoch 34/40
16/16 [==============================] - 0s 2ms/step - loss: 0.6848 - mean_absolute_error: 0.6848 - val_loss: 0.1118 - val_mean_absolute_error: 0.1118
Epoch 35/40
16/16 [==============================] - 0s 2ms/step - loss: 0.6542 - mean_absolute_error: 0.6542 - val_loss: 0.1127 - val_mean_absolute_error: 0.1127
Epoch 36/40
16/16 [===

16/16 [==============================] - 0s 2ms/step - loss: 0.9217 - mean_absolute_error: 0.9217 - val_loss: 0.1523 - val_mean_absolute_error: 0.1523
Epoch 41/80
16/16 [==============================] - 0s 2ms/step - loss: 0.7034 - mean_absolute_error: 0.7034 - val_loss: 0.1474 - val_mean_absolute_error: 0.1474
Epoch 42/80
16/16 [==============================] - 0s 1ms/step - loss: 0.6626 - mean_absolute_error: 0.6626 - val_loss: 0.1460 - val_mean_absolute_error: 0.1460
Epoch 43/80
16/16 [==============================] - 0s 2ms/step - loss: 0.9177 - mean_absolute_error: 0.9177 - val_loss: 0.1498 - val_mean_absolute_error: 0.1498
Epoch 44/80
16/16 [==============================] - 0s 2ms/step - loss: 1.0964 - mean_absolute_error: 1.0964 - val_loss: 0.1536 - val_mean_absolute_error: 0.1536
Epoch 45/80
16/16 [==============================] - 0s 3ms/step - loss: 1.0071 - mean_absolute_error: 1.0071 - val_loss: 0.1525 - val_mean_absolute_error: 0.1525
Epoch 46/80
16/16 [===============

16/16 [==============================] - 0s 2ms/step - loss: 0.6117 - mean_absolute_error: 0.6117 - val_loss: 0.1486 - val_mean_absolute_error: 0.1486
Epoch 11/120
16/16 [==============================] - 0s 2ms/step - loss: 0.7751 - mean_absolute_error: 0.7751 - val_loss: 0.1456 - val_mean_absolute_error: 0.1456
Epoch 12/120
16/16 [==============================] - 0s 1ms/step - loss: 0.8172 - mean_absolute_error: 0.8172 - val_loss: 0.1423 - val_mean_absolute_error: 0.1423
Epoch 13/120
16/16 [==============================] - 0s 2ms/step - loss: 0.7194 - mean_absolute_error: 0.7194 - val_loss: 0.1436 - val_mean_absolute_error: 0.1436
Epoch 14/120
16/16 [==============================] - 0s 2ms/step - loss: 0.7504 - mean_absolute_error: 0.7504 - val_loss: 0.1397 - val_mean_absolute_error: 0.1397
Epoch 15/120
16/16 [==============================] - 0s 2ms/step - loss: 0.7373 - mean_absolute_error: 0.7373 - val_loss: 0.1346 - val_mean_absolute_error: 0.1346
Epoch 16/120
16/16 [=========

Epoch 60/120
16/16 [==============================] - 0s 2ms/step - loss: 0.4990 - mean_absolute_error: 0.4990 - val_loss: 0.1366 - val_mean_absolute_error: 0.1366
Epoch 61/120
16/16 [==============================] - 0s 2ms/step - loss: 0.7904 - mean_absolute_error: 0.7904 - val_loss: 0.1364 - val_mean_absolute_error: 0.1364
Epoch 62/120
16/16 [==============================] - 0s 2ms/step - loss: 0.6611 - mean_absolute_error: 0.6611 - val_loss: 0.1366 - val_mean_absolute_error: 0.1366
Epoch 63/120
16/16 [==============================] - 0s 1ms/step - loss: 1.1050 - mean_absolute_error: 1.1050 - val_loss: 0.1317 - val_mean_absolute_error: 0.1317
Epoch 64/120
16/16 [==============================] - 0s 2ms/step - loss: 0.8395 - mean_absolute_error: 0.8395 - val_loss: 0.1364 - val_mean_absolute_error: 0.1364
Epoch 65/120
16/16 [==============================] - 0s 1ms/step - loss: 0.6847 - mean_absolute_error: 0.6847 - val_loss: 0.1354 - val_mean_absolute_error: 0.1354
Epoch 66/120
16/

Epoch 110/120
16/16 [==============================] - 0s 1ms/step - loss: 0.8041 - mean_absolute_error: 0.8041 - val_loss: 0.1290 - val_mean_absolute_error: 0.1290
Epoch 111/120
16/16 [==============================] - 0s 2ms/step - loss: 0.6689 - mean_absolute_error: 0.6689 - val_loss: 0.1234 - val_mean_absolute_error: 0.1234
Epoch 112/120
16/16 [==============================] - 0s 1ms/step - loss: 0.7526 - mean_absolute_error: 0.7526 - val_loss: 0.1229 - val_mean_absolute_error: 0.1229
Epoch 113/120
16/16 [==============================] - 0s 2ms/step - loss: 0.4447 - mean_absolute_error: 0.4447 - val_loss: 0.1258 - val_mean_absolute_error: 0.1258
Epoch 114/120
16/16 [==============================] - 0s 2ms/step - loss: 0.6095 - mean_absolute_error: 0.6095 - val_loss: 0.1275 - val_mean_absolute_error: 0.1275
Epoch 115/120
16/16 [==============================] - 0s 1ms/step - loss: 0.7017 - mean_absolute_error: 0.7017 - val_loss: 0.1320 - val_mean_absolute_error: 0.1320
Epoch 116/

In [443]:
m1.predict(X_test)

array([[-0.11704713],
       [ 0.21536821],
       [-0.04705172],
       [-0.13012855]], dtype=float32)

In [442]:
np.array(y_test)

array([-0.18464182, -0.05794234, -0.02727273,  0.02902711])